# 🚀 Proyecto Integrador 2: Pipeline Near Real-Time, Scheduling y Alertas

Este proyecto consolida el nivel Junior con un pipeline que monitorea una carpeta de entrada, valida y transforma nuevos archivos CSV en tiempo casi real, carga resultados, genera métricas, registra logs, implementa reintentos con backoff y envía notificaciones (stub).

Objetivos:
- ✅ Monitoreo de archivos nuevos (file watcher)
- ✅ Validación de esquema y calidad
- ✅ Transformación y enriquecimiento
- ✅ Carga a SQLite y Parquet
- ✅ Métricas y logging estructurado
- ✅ Reintentos con backoff e idempotencia
- ✅ Notificaciones (Slack/Email stub)
- ✅ Scheduling simple sin dependencias externas

## 1. Configuración y Estructura

In [ ]:
import os, json, time, sqlite3, logging, traceback, hashlib
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import pandas as pd
import numpy as np

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
INPUT_DIR = os.path.join(BASE_DIR, 'ingest', 'incoming')
CHKPT_DIR = os.path.join(BASE_DIR, 'ingest', 'checkpoints')
OUT_DIR = os.path.join(BASE_DIR, 'outputs', 'proyecto_2')
LOGS_DIR = os.path.join(BASE_DIR, 'logs')

for d in [INPUT_DIR, CHKPT_DIR, OUT_DIR, LOGS_DIR]:
    os.makedirs(d, exist_ok=True)

DB_PATH = os.path.join(OUT_DIR, 'near_rt_analytics.db')
PARQUET_DIR = os.path.join(OUT_DIR, 'parquet')
os.makedirs(PARQUET_DIR, exist_ok=True)

# Logging
logging.basicConfig(level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(LOGS_DIR, 'proyecto_integrador_2.log'), encoding='utf-8'),
        logging.StreamHandler()
    ])
logger = logging.getLogger('near_rt_pipeline')
logger.info('✅ Entorno inicializado')
print('BASE_DIR:', BASE_DIR)
print('INPUT_DIR:', INPUT_DIR)
print('OUT_DIR:', OUT_DIR)

### 🏗️ Arquitectura de Pipeline Near Real-Time

**Concepto:** Pipeline que procesa archivos automáticamente al detectar nuevos en carpeta monitoreada.

**Estructura de directorios:**
- **ingest/incoming/**: Archivos nuevos llegan aquí (trigger)
- **ingest/checkpoints/**: Metadata de archivos procesados (evita duplicados)
- **outputs/proyecto_2/**: Datos transformados (DB + Parquet)
- **logs/**: Trazabilidad de ejecución

**Patrón:**
1. Watchdog monitorea incoming/
2. Detecta nuevo archivo → valida → transforma → carga
3. Guarda checkpoint para idempotencia

**Uso:** Pipelines batch-streaming, ingesta continua sin colas MQ complejas.

## 2. Utilidades: Hash, Checkpoints, Notificaciones (stub)

In [ ]:
def file_md5(path: str) -> str:
    h = hashlib.md5()
    with open(path, 'rb') as f:
        for chunk in iter(lambda: f.read(8192), b''):
            h.update(chunk)
    return h.hexdigest()

def chkpt_path(filename: str) -> str:
    base = os.path.basename(filename)
    return os.path.join(CHKPT_DIR, base + '.json')

def is_processed(filename: str, md5sum: str) -> bool:
    p = chkpt_path(filename)
    if not os.path.exists(p):
        return False
    try:
        data = json.load(open(p, 'r', encoding='utf-8'))
        return data.get('md5') == md5sum
    except Exception:
        return False

def save_chkpt(filename: str, md5sum: str, rows: int, status: str, error: Optional[str]=None):
    p = chkpt_path(filename)
    payload = {
        'filename': os.path.basename(filename),
        'md5': md5sum,
        'rows': rows,
        'status': status,
        'error': error,
        'timestamp': datetime.utcnow().isoformat()
    }
    json.dump(payload, open(p, 'w', encoding='utf-8'), ensure_ascii=False, indent=2)

def notify(message: str, level: str = 'info') -> None:
    """
    Stub de notificación: aquí iría Slack webhook o email SMTP.
    Para Slack: POST a https://hooks.slack.com/services/... con {'text': message}
    Para Email: usar smtplib con credenciales seguras (no hardcodear).
    """
    levels = {'info': logging.INFO, 'warning': logging.WARNING, 'error': logging.ERROR}
    logger.log(levels.get(level, logging.INFO), f'🔔 {message}')

print('✅ Utilidades listas')

### 🔒 Idempotencia: Checkpoints y Hash MD5

**Concepto:** Evitar procesar el mismo archivo múltiples veces mediante checksums y metadata.

**Componentes:**
- **MD5 Hash:** Firma única del contenido del archivo
- **Checkpoint JSON:** Almacena {filename, md5, timestamp, status}
- **is_processed():** Verifica si archivo ya fue procesado

**Flujo:**
1. Calcular MD5 del archivo nuevo
2. Buscar checkpoint existente
3. Si MD5 coincide → skip (ya procesado)
4. Si no → procesar y guardar checkpoint

**Importancia:** Pipeline idempotente permite reintentos sin duplicar datos (requisito producción).

## 3. Validación de Esquema y Transformación

In [ ]:
EXPECTED_COLUMNS = {
    'venta_id': 'int64',
    'cliente_id': 'int64',
    'producto_id': 'int64',
    'cantidad': 'int64',
    'precio_unitario': 'float64',
    'total': 'float64',
    'fecha_venta': 'datetime64[ns]',
    'metodo_pago': 'object',
    'estado': 'object'
}

def read_csv_safe(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    # Normalización mínima
    if 'fecha_venta' in df.columns:
        df['fecha_venta'] = pd.to_datetime(df['fecha_venta'], errors='coerce')
    return df

def validate_schema(df: pd.DataFrame) -> Tuple[bool, List[str]]:
    errors = []
    for col, dtype in EXPECTED_COLUMNS.items():
        if col not in df.columns:
            errors.append(f
        else:
            # Validación de tipo (suave)
            if dtype.startswith('datetime'):
                if not np.issubdtype(df[col].dtype, np.datetime64):
                    errors.append(f"Tipo inválido en {col}: {df[col].dtype}")
            elif dtype == 'object':
                pass
            else:
                if str(df[col].dtype) != dtype:
                    errors.append(f"Tipo inválido en {col}: {df[col].dtype} (esperado {dtype})")
    return (len(errors) == 0, errors)

def transform(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['anio'] = out['fecha_venta'].dt.year
    out['mes'] = out['fecha_venta'].dt.to_period('M').astype(str)
    out['ticket_unitario'] = out['total'] / out['cantidad']
    return out

print('✅ Validadores definidos')

### ✓ Validación de Esquema: Data Contract

**Concepto:** Validar que datos entrantes cumplan esquema esperado antes de procesar.

**Data Contract:**
- Diccionario con columnas esperadas y tipos de datos
- Valida presencia de columnas obligatorias
- Verifica tipos de datos (int64, float64, datetime64, object)

**Flujo:**
1. Leer CSV con `pd.read_csv()`
2. Normalizar fechas con `pd.to_datetime(errors='coerce')`
3. Validar esquema contra contrato
4. Si falla → logear errores y rechazar archivo
5. Si pasa → aplicar transformaciones

**Ventaja:** Detección temprana de problemas, evita corrupción downstream.

## 4. Carga y Métricas

In [ ]:
def load_to_sqlite(df: pd.DataFrame, table: str = 'ventas_near_rt') -> None:
    conn = sqlite3.connect(DB_PATH)
    try:
        df.to_sql(table, conn, if_exists='append', index=False)
    finally:
        conn.close()

def export_parquet(df: pd.DataFrame, base_name: str) -> str:
    ts = datetime.utcnow().strftime('%Y%m%dT%H%M%S')
    out_path = os.path.join(PARQUET_DIR, f
)
    df.to_parquet(out_path, index=False)
    return out_path

def compute_metrics(df: pd.DataFrame) -> Dict:
    return {
        'rows': len(df),
        'total_sum': float(df['total'].sum()),
        'ticket_promedio': float(df['total'].mean()),
        'clientes_unicos': int(df['cliente_id'].nunique())
    }

print('✅ Funciones de carga y métricas listas')

### 💾 Dual Output: SQLite + Parquet

**Concepto:** Persistir datos en múltiples formatos para diferentes casos de uso.

**SQLite:**
- `if_exists='append'`: añade registros sin sobrescribir
- Ideal para: queries SQL, análisis ad-hoc, dashboards pequeños
- Limitación: no escala para Big Data

**Parquet:**
- Formato columnar comprimido
- Timestamped: `ventas_20250101T120000.parquet`
- Ideal para: data lakes, analytics distribuidos (Spark, Athena)
- Ventaja: 10-100x más eficiente que CSV

**Estrategia:** SQLite para operacional, Parquet para histórico y analytics.

## 5. Worker con Reintentos y Backoff

In [ ]:
def process_file(path: str, max_retries: int = 3) -> bool:
    md5sum = file_md5(path)
    if is_processed(path, md5sum):
        logger.info(f'⏭️ Ya procesado: {os.path.basename(path)}')
        return True

    sleep = 1.0
    for attempt in range(1, max_retries+1):
        try:
            logger.info(f'📥 Procesando {os.path.basename(path)} (intento {attempt})')
            df = read_csv_safe(path)
            ok, errs = validate_schema(df)
            if not ok:
                raise ValueError('Esquema inválido: ' + '; '.join(errs))
            df_t = transform(df)
            load_to_sqlite(df_t)
            pq = export_parquet(df_t, base_name=os.path.splitext(os.path.basename(path))[0])
            m = compute_metrics(df_t)
            save_chkpt(path, md5sum, rows=len(df_t), status='ok')
            notify(f

### 🔄 Retry Logic con Exponential Backoff

**Concepto:** Reintentar operaciones fallidas con delays incrementales para manejar errores transitorios.

**Parámetros:**
- **max_retries:** número máximo de intentos (ej: 3)
- **base_delay:** delay inicial en segundos (ej: 2s)
- **backoff:** multiplicador entre intentos (ej: 2x)

**Ejemplo de delays:**
- Intento 1: 2s
- Intento 2: 4s (2 × 2)
- Intento 3: 8s (4 × 2)

**Cuándo usar:**
- Errores de red/conexión
- Rate limits de APIs
- Locks de base de datos
- Servicios externos temporalmente caídos

**Nota:** No reintentar errores lógicos (validación fallida, datos corruptos).

## 6. Scheduler Simple con Polling

In [ ]:
def scan_and_process(interval_sec: int = 5, max_iterations: int = 10) -> None:
    """
    Monitorea carpeta incoming/ cada interval_sec segundos.
    Procesa archivos nuevos detectados.
    """
    logger.info(f'👀 Iniciando monitoreo cada {interval_sec}s (max {max_iterations} iteraciones)')
    
    for i in range(max_iterations):
        logger.info(f'🔍 Scan #{i+1}')
        files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.csv')]
        
        if not files:
            logger.info('  └─ No hay archivos CSV')
        else:
            logger.info(f'  └─ Detectados {len(files)} archivos')
            for fname in files:
                fpath = os.path.join(INPUT_DIR, fname)
                success = process_file(fpath, max_retries=3)
                
                # Mover archivo procesado a carpeta processed/
                processed_dir = os.path.join(BASE_DIR, 'ingest', 'processed')
                os.makedirs(processed_dir, exist_ok=True)
                dest = os.path.join(processed_dir, fname)
                
                try:
                    os.rename(fpath, dest)
                    logger.info(f'  ✅ Movido a processed/: {fname}')
                except Exception as e:
                    logger.warning(f'  ⚠️ No se pudo mover {fname}: {e}')
        
        if i < max_iterations - 1:
            time.sleep(interval_sec)
    
    logger.info('🏁 Monitoreo finalizado')

print('✅ Función de scheduler lista')

### ⏰ Polling Loop: Monitoreo Periódico

**Concepto:** Revisar carpeta en intervalos regulares para detectar nuevos archivos.

**Implementación:**
- Loop con `time.sleep(interval_sec)`
- Lista archivos `.csv` en `incoming/`
- Procesa cada uno con `process_file()`
- Mueve archivos a `processed/` después de procesar

**Ventajas:**
- Simple, sin dependencias externas
- Fácil de debuggear
- Suficiente para volúmenes bajos/medios

**Desventajas:**
- No es real-time (delay = interval)
- Consumo CPU innecesario si no hay archivos
- No escala para miles de archivos

**Alternativas avanzadas:**
- **watchdog**: Library Python para file system events
- **Airflow FileSensor**: Detecta archivos en schedule
- **Kafka/Pub-Sub**: Event streaming para pipelines distribuidos

**Uso:** Prototipos, demos, ambientes controlados.

---

## 7. Ejecución y Testing

Generaremos archivos de prueba y ejecutaremos el pipeline para verificar su funcionamiento.

### 🧪 Generar Archivos de Prueba

In [ ]:
import random
from datetime import datetime, timedelta

def generate_test_files(num_files: int = 3, rows_per_file: int = 10) -> None:
    """Genera archivos CSV de prueba en la carpeta incoming."""
    productos = ["Laptop", "Mouse", "Teclado", "Monitor", "Webcam", "Auriculares"]
    categorias = ["Electrónica", "Accesorios"]
    
    for i in range(num_files):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = INPUT_DIR / f"ventas_{timestamp}_{i}.csv"
        
        rows = []
        base_date = datetime.now() - timedelta(days=random.randint(1, 30))
        
        for j in range(rows_per_file):
            fecha = (base_date + timedelta(hours=j)).strftime("%Y-%m-%d %H:%M:%S")
            producto = random.choice(productos)
            categoria = random.choice(categorias)
            cantidad = random.randint(1, 5)
            precio = round(random.uniform(10.0, 500.0), 2)
            
            # Introducir errores aleatorios (10% de probabilidad)
            if random.random() < 0.1:
                # Error de tipo: cantidad negativa
                cantidad = -cantidad
            
            rows.append({
                "fecha": fecha,
                "producto": producto,
                "categoria": categoria,
                "cantidad": cantidad,
                "precio": precio
            })
        
        df = pd.DataFrame(rows)
        df.to_csv(filename, index=False)
        print(f"✅ Generado: {filename.name} ({len(df)} filas)")
        
        # Esperar un poco para que los timestamps sean diferentes
        time.sleep(0.5)
    
    print(f"\n📁 Total archivos generados: {num_files}")

# Generar archivos de prueba
generate_test_files(num_files=3, rows_per_file=10)

### ▶️ Ejecutar el Pipeline

In [ ]:
# Ejecutar el scheduler (procesará los archivos generados)
print("🚀 Iniciando pipeline...")
print(f"📂 Monitoreando: {INPUT_DIR}")
print(f"⏱️  Intervalo: 5 segundos")
print(f"🔄 Max iteraciones: 10\n")

scan_and_process(interval_sec=5, max_iterations=10)

print("\n✅ Pipeline completado")

### 📊 Verificar Resultados en Base de Datos

In [ ]:
import sqlite3

# Conectar a la base de datos
conn = sqlite3.connect(DB_PATH)

# Verificar registros insertados
print("📊 DATOS EN LA BASE DE DATOS:")
print("=" * 60)

query = """
SELECT 
    fecha,
    producto,
    categoria,
    cantidad,
    precio,
    precio * cantidad as total
FROM ventas
ORDER BY fecha DESC
LIMIT 20
"""

df_results = pd.read_sql_query(query, conn)
print(f"\nTotal registros en DB: {len(pd.read_sql_query('SELECT * FROM ventas', conn))}")
print(f"\nÚltimos 20 registros:\n")
print(df_results.to_string(index=False))

# Estadísticas por producto
print("\n\n📈 ESTADÍSTICAS POR PRODUCTO:")
print("=" * 60)

stats_query = """
SELECT 
    producto,
    COUNT(*) as total_ventas,
    SUM(cantidad) as unidades_vendidas,
    ROUND(AVG(precio), 2) as precio_promedio,
    ROUND(SUM(precio * cantidad), 2) as revenue_total
FROM ventas
GROUP BY producto
ORDER BY revenue_total DESC
"""

df_stats = pd.read_sql_query(stats_query, conn)
print(df_stats.to_string(index=False))

conn.close()

---

## 8. Métricas y Auditoría

Verificaremos los checkpoints para auditar el procesamiento de archivos.

### 📋 Revisión de Checkpoints

In [ ]:
# Leer checkpoints para auditoría
if CHECKPOINT_FILE.exists():
    df_checkpoints = pd.read_csv(CHECKPOINT_FILE)
    
    print("📋 HISTORIAL DE PROCESAMIENTO:")
    print("=" * 80)
    print(df_checkpoints.to_string(index=False))
    
    # Resumen de estados
    print("\n\n📊 RESUMEN POR ESTADO:")
    print("=" * 80)
    status_summary = df_checkpoints['status'].value_counts()
    for status, count in status_summary.items():
        icon = "✅" if status == "success" else "❌"
        print(f"{icon} {status}: {count}")
    
    # Métricas de tiempo
    print("\n\n⏱️  MÉTRICAS DE TIEMPO:")
    print("=" * 80)
    df_checkpoints['duration'] = pd.to_datetime(df_checkpoints['processed_at']) - pd.to_datetime(df_checkpoints['processed_at'])
    
    success_files = df_checkpoints[df_checkpoints['status'] == 'success']
    if len(success_files) > 0:
        print(f"Total archivos procesados exitosamente: {len(success_files)}")
        print(f"Total registros insertados: {success_files['records_inserted'].sum()}")
        print(f"Promedio registros por archivo: {success_files['records_inserted'].mean():.1f}")
    
    failed_files = df_checkpoints[df_checkpoints['status'] == 'failed']
    if len(failed_files) > 0:
        print(f"\n⚠️  Archivos fallidos: {len(failed_files)}")
        print("\nDetalles de errores:")
        for _, row in failed_files.iterrows():
            print(f"  - {row['filename']}: {row['error_message']}")
else:
    print("⚠️  No se encontró archivo de checkpoints")

### 🗂️ Verificar Archivos Movidos

In [ ]:
# Verificar que los archivos se movieron correctamente
print("📂 ESTADO DE LAS CARPETAS:")
print("=" * 60)

incoming_files = list(INPUT_DIR.glob("*.csv"))
processed_files = list(PROCESSED_DIR.glob("*.csv"))
failed_files = list(FAILED_DIR.glob("*.csv"))

print(f"\n📥 Incoming: {len(incoming_files)} archivos")
for f in incoming_files:
    print(f"  - {f.name}")

print(f"\n✅ Processed: {len(processed_files)} archivos")
for f in processed_files:
    print(f"  - {f.name}")

print(f"\n❌ Failed: {len(failed_files)} archivos")
for f in failed_files:
    print(f"  - {f.name}")

print("\n💡 Resultado esperado: Incoming debe estar vacío después del procesamiento")

---

## 9. Conclusión y Siguientes Pasos

### 🎯 Lo que Hemos Construido

En este proyecto integrador hemos desarrollado un **pipeline de datos casi en tiempo real** que incluye:

1. **Configuración modular**: Paths centralizados y fácilmente configurables
2. **Validación robusta**: Esquemas estrictos con manejo de errores detallado
3. **Idempotencia**: Checkpoints para evitar reprocesamiento
4. **Retry logic**: Reintentos automáticos con backoff exponencial
5. **Scheduler simple**: Polling loop para monitoreo continuo
6. **Auditoría completa**: Tracking de cada archivo procesado
7. **Gestión de archivos**: Organización automática (incoming → processed/failed)

### 🚀 Componentes Clave del Pipeline

```
incoming/           →  Archivos nuevos llegan aquí
   ↓
[Scheduler]        →  Detecta archivos cada N segundos
   ↓
[Validation]       →  Verifica esquema y calidad de datos
   ↓
[Transform]        →  Limpieza y preparación
   ↓
[Load]             →  Inserción a SQLite con retry
   ↓
[Checkpoint]       →  Registro de auditoría
   ↓
processed/failed/  →  Organización final
```

### 💡 Conceptos Aplicados

- **ETL (Extract-Transform-Load)**: Extracción de CSV, transformación con Pandas, carga a SQL
- **Data Quality**: Validación de tipos, rangos, valores requeridos
- **Resilience**: Manejo de errores, reintentos, estados de recuperación
- **Near Real-Time**: Latencia de segundos con polling loop
- **Observability**: Logs, checkpoints, métricas de procesamiento

### 📊 Métricas de Éxito

Un pipeline bien diseñado debe cumplir:

✅ **Corrección**: 100% de registros válidos insertados  
✅ **Idempotencia**: Sin duplicados en reprocesamiento  
✅ **Trazabilidad**: Auditoría completa de cada archivo  
✅ **Resiliencia**: Recuperación automática de errores transitorios  
✅ **Eficiencia**: Procesamiento en segundos para volúmenes bajos/medios  

### 🔧 Mejoras Futuras

**Nivel Mid:**
1. **Apache Airflow**: Reemplazar polling con DAGs programados
2. **Validación avanzada**: Great Expectations para data quality
3. **Particionamiento**: Organizar datos por fecha para queries eficientes
4. **Paralelización**: Procesar múltiples archivos simultáneamente
5. **Alertas**: Notificaciones email/Slack en caso de errores

**Nivel Senior:**
6. **Streaming real**: Kafka + Spark Structured Streaming para latencia <1s
7. **Data Lake**: Almacenamiento en S3/GCS con formato Parquet/Delta
8. **Linaje de datos**: Rastreo completo del origen de cada registro
9. **ML Integration**: Feature store para modelos de ML
10. **Cloud deployment**: Kubernetes + auto-scaling

### 🎓 Habilidades Desarrolladas

- ✅ Diseño de ETL incremental
- ✅ Validación de datos con esquemas
- ✅ Manejo de errores y reintentos
- ✅ Logging y auditoría
- ✅ Patrones de resiliencia
- ✅ Organización de código modular
- ✅ Testing con datos sintéticos

### 📚 Recursos Adicionales

- **Great Expectations**: https://greatexpectations.io/
- **Apache Airflow**: https://airflow.apache.org/
- **Data Quality Patterns**: https://www.thoughtworks.com/insights/blog/data-quality
- **Python Logging Best Practices**: https://docs.python.org/3/howto/logging.html

---

### 🏆 ¡Felicidades!

Has completado el **Proyecto Integrador 2** del nivel Junior. Este pipeline es la base de sistemas de datos profesionales.

**Próximo paso:** Nivel Mid → Aprende Airflow, Cloud (AWS/GCP/Azure), y arquitecturas distribuidas 🚀